## Part 2

 (5 pts) Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):

(10 pts) One problem with estimating the emission parameters is that some words that appear in the
test set do not appear in the training set. One simple idea to handle this issue is as follows. First,
replace those words that appear less than k times in the training set with a special token #UNK#
before training. This leads to a “modified training set”. We then use such a modified training set to
train our model.
During the testing phase, if the word does not appear in the “modified training set”, we replace that
word with #UNK# as well.
Set k to 3, implement this fix into your function for computing the emission parameters.

(10 pts) Implement a simple sentiment analysis system that produces the tag
y
∗ = arg max
y
e(x|y)
for each word x in the sequence.
For all the four datasets EN, FR, CN, and SG, learn these parameters with train, and evaluate your
system on the development set dev.in for each of the dataset. Write your output to dev.p2.out
for the four datasets respectively. Compare your outputs and the gold-standard outputs in dev.out
and report the precision, recall and F scores of such a baseline system for each dataset.
The precision score is defined as follows:
Precision =
Total number of correctly predicted entities
Total number of predicted entities
The recall score is defined as follows:
Recall =
Total number of correctly predicted entities
Total number of gold entities
where a gold entity is a true entity that is annotated in the reference output file, and a predicted entity
is regarded as correct if and only if it matches exactly the gold entity (i.e., both their boundaries and
sentiment are exactly the same).
4
Finally the F score is defined as follows:
F =
2
1/Precision + 1/Recall
Note: in some cases, you might have an output sequence that consists of a transition from O to
I-negative (rather than B-negative). For example, “O I-negative I-negative O”.
In this case, the second and third words should be regarded as one entity with negative sentiment.
You can use the evaluation script shared with you to calculate such scores. However it is strongly
encouraged that you understand how the scores are calculated.

In [16]:
import numpy as np
import requests

r = open('train', "r")
output = []
entities = []

o_count =0
b_positive=0
b_neutral=0
b_negative=0

for l in r:
    line = []
    if l != "\n":
        output.append(tuple(l.strip("\n").split(" ")))

        
entity=[]
for o in output:
    
    if o[1][0] == 'B':
        if o[1] == 'B-neutral':
            b_neutral += 1
        if o[1] == 'B-positive':
            b_positive += 1
        if o[1] == 'B-negative':
            b_negative += 1
    if o[1] == 'O':
        o_count += 1

print(b_positive)
print(b_neutral)
print(b_negative)
print(o_count)

output

1208
65
382
24242


[('We', 'O'),
 ('were', 'O'),
 ('then', 'O'),
 ('charged', 'O'),
 ('for', 'O'),
 ('their', 'O'),
 ('most', 'O'),
 ('expensive', 'O'),
 ('sake', 'O'),
 ('(', 'O'),
 ('$', 'O'),
 ('20', 'O'),
 ('+', 'O'),
 ('per', 'O'),
 ('serving', 'O'),
 (')', 'O'),
 ('when', 'O'),
 ('we', 'O'),
 ('in', 'O'),
 ('fact', 'O'),
 ('drank', 'O'),
 ('a', 'O'),
 ('sake', 'O'),
 ('of', 'O'),
 ('less', 'O'),
 ('than', 'O'),
 ('half', 'O'),
 ('that', 'O'),
 ('price', 'O'),
 ('.', 'O'),
 ('Amma', 'B-neutral'),
 ('is', 'O'),
 ('nothing', 'O'),
 ('special', 'O'),
 ('.', 'O'),
 ('Great', 'O'),
 ('atmoshere', 'B-positive'),
 ('and', 'O'),
 ('worth', 'O'),
 ('every', 'O'),
 ('bit', 'O'),
 ('.', 'O'),
 ('delicious', 'O'),
 ('bagels', 'B-positive'),
 (',', 'O'),
 ('especially', 'O'),
 ('when', 'O'),
 ('right', 'O'),
 ('out', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('oven', 'O'),
 ('.', 'O'),
 ('Go', 'O'),
 ('here', 'O'),
 ('.', 'O'),
 ('Highly', 'O'),
 ('recommended', 'O'),
 ('!', 'O'),
 ('But', 'O'),
 ('who', 'O'),
 ('says

In [18]:
words = {}
tags = {}
tag_emits_word = {}

for pair in output:
    if pair[1] in tags:
        tags[pair[1]] += 1
    else:
        tags[pair[1]] = 1
        
    if pair[0] in words:
        words[pair[0]] += 1
    else:
        words[pair[0]] = 1
    
    if pair in tag_emits_word:
        tag_emits_word[pair] +=1
    else:
        tag_emits_word[pair] =1
        
tag_emits_word

emissionParams = {}

for tag_word_pair in tag_emits_word:
    if tag_word_pair not in emissionParams:
        tag_word_pair
        
        
    
    

{('We', 'O'): 83,
 ('were', 'O'): 125,
 ('then', 'O'): 15,
 ('charged', 'O'): 2,
 ('for', 'O'): 279,
 ('their', 'O'): 52,
 ('most', 'O'): 26,
 ('expensive', 'O'): 13,
 ('sake', 'O'): 6,
 ('(', 'O'): 80,
 ('$', 'O'): 46,
 ('20', 'O'): 6,
 ('+', 'O'): 4,
 ('per', 'O'): 5,
 ('serving', 'O'): 3,
 (')', 'O'): 83,
 ('when', 'O'): 41,
 ('we', 'O'): 133,
 ('in', 'O'): 256,
 ('fact', 'O'): 9,
 ('drank', 'O'): 1,
 ('a', 'O'): 537,
 ('of', 'O'): 296,
 ('less', 'O'): 6,
 ('than', 'O'): 38,
 ('half', 'O'): 9,
 ('that', 'O'): 179,
 ('price', 'O'): 28,
 ('.', 'O'): 1524,
 ('Amma', 'B-neutral'): 1,
 ('is', 'O'): 462,
 ('nothing', 'O'): 17,
 ('special', 'O'): 14,
 ('Great', 'O'): 30,
 ('atmoshere', 'B-positive'): 1,
 ('and', 'O'): 765,
 ('worth', 'O'): 31,
 ('every', 'O'): 23,
 ('bit', 'O'): 19,
 ('delicious', 'O'): 53,
 ('bagels', 'B-positive'): 5,
 (',', 'O'): 943,
 ('especially', 'O'): 9,
 ('right', 'O'): 8,
 ('out', 'O'): 58,
 ('the', 'O'): 1101,
 ('oven', 'O'): 1,
 ('Go', 'O'): 9,
 ('here', 'O'): 